In [6]:
%load_ext autoreload
%autoreload 2
import sys
sys.executable

import os
import sys
import numpy as np
import pandas as pd
import datetime
import seaborn as sns
import ogb
from tqdm import tqdm
import hiplot as hip
from copy import deepcopy
import datetime
import json
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch.utils.data import Subset, TensorDataset
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw

print(f"{torch.__version__=}")
print(f"{torch.__file__=}")
print(f"{torch.cuda.device_count()=}")
print(f"{torch.cuda.is_available()=}")
print(f"{torch.version.cuda=}")



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
torch.__version__='2.0.0+cu117'
torch.__file__='/home/ubuntu/anaconda3/envs/bedict_crispr/lib/python3.10/site-packages/torch/__init__.py'
torch.cuda.device_count()=1
torch.cuda.is_available()=True
torch.version.cuda='11.7'


In [7]:
from torch_geometric.data import Data

In [8]:
cwd = os.getcwd()
print(cwd)
cwd_parent = os.path.abspath(os.path.join(cwd, os.pardir))
print(cwd_parent)

sys.path.append(cwd_parent)

import deepadr
from deepadr.dataset import *
from deepadr.utilities import *
from deepadr.chemfeatures import *
from deepadr.train_functions import *
from deepadr.model_gnn_ogb import GNN, DeepAdr_SiameseTrf, ExpressionNN

/home/ubuntu/mnt/graphnn/notebooks
/home/ubuntu/mnt/graphnn


In [9]:
import sys
print("Python version")
print (sys.version)

Python version
3.10.9 (main, Jan 11 2023, 15:21:40) [GCC 11.2.0]


In [10]:
rawdata_dir = '../data/raw/'
processed_dir = '../data/processed/'
up_dir = '..'

In [11]:
report_available_cuda_devices()
n_gpu = torch.cuda.device_count()
n_gpu
device_cpu = get_device(to_gpu=True)
# device_gpu = get_device(True, index=0)
print("torch:", torch.__version__)
print("CUDA:", torch.version.cuda)
print(sys.version)

number of GPUs available: 1
cuda:0, name:Tesla T4
total memory available: 14.755615234375 GB
total memory allocated on device: 0.0 GB
max memory allocated on device: 0.0 GB
total memory cached on device: 0.0 GB
max memory cached  on device: 0.0 GB

torch: 2.0.0+cu117
CUDA: 11.7
3.10.9 (main, Jan 11 2023, 15:21:40) [GCC 11.2.0]


In [12]:
device_cpu

device(type='cuda', index=0)

### Preparing dataset 

In [13]:
# options: 
# 'total_thresh' + 4,3,2
# 'loewe_thresh', 'hsa_thresh', 'bliss_thresh', 'zip_thresh' + 1

score = 'bliss_thresh'
score_val = 1

In [14]:
DSdataset_name = f'DrugComb_{score}_{score_val}'
data_fname = 'data_v1' # v2 for baseline models, v3 for additive samples
DSdataset_name

'DrugComb_bliss_thresh_1'

In [15]:
targetdata_dir = create_directory(os.path.join(processed_dir, DSdataset_name, data_fname))
targetdata_dir_raw = create_directory(os.path.join(targetdata_dir, "raw"))
targetdata_dir_processed = create_directory(os.path.join(targetdata_dir, "processed"))
targetdata_dir_exp = create_directory(os.path.join(targetdata_dir, "experiments"))
# # ReaderWriter.dump_data(dpartitions, os.path.join(targetdata_dir, 'data_partitions.pkl'))
print(targetdata_dir)

/home/ubuntu/mnt/graphnn/data/processed/DrugComb_bliss_thresh_1/data_v1


In [16]:
%%time
# Make sure to first run the "DDoS_Dataset_Generation" notebook first
dataset = MoleculeDataset(root=targetdata_dir)

CPU times: user 0 ns, sys: 391 ms, total: 391 ms
Wall time: 508 ms


In [17]:
import pickle
with open(targetdata_dir_raw +'/data_pairs.pkl', 'rb') as file:
    data_pairs = pickle.load(file )

In [18]:
data_pairs

,Drug1_ID,Drug2_ID,Cell_Line_ID,Cosmic_ID,Drug1,Drug2,Y
0,Ethyl bromopyruvate,717906-29-1,A-673,684052,CCOC(=O)C(=O)CBr,CN(C1=CC=CC=C1CNC2=NC(=NC=C2C(F)(F)F)NC3=CC4=C...,0
1,dacarbazine,717906-29-1,A-673,684052,CN(C)N=NC1=C(NC=N1)C(=O)N,CN(C1=CC=CC=C1CNC2=NC(=NC=C2C(F)(F)F)NC3=CC4=C...,0
2,chlorambucil,717906-29-1,A-673,684052,C1=CC(=CC=C1CCCC(=O)O)N(CCCl)CCCl,CN(C1=CC=CC=C1CNC2=NC(=NC=C2C(F)(F)F)NC3=CC4=C...,0
3,Nedaplatin,717906-29-1,A-673,684052,C(C(=O)O)O.N.N.[Pt],CN(C1=CC=CC=C1CNC2=NC(=NC=C2C(F)(F)F)NC3=CC4=C...,0
4,colchicine,717906-29-1,A-673,684052,CC(=O)NC1CCC2=CC(=C(C(=C2C3=CC=C(C(=O)C=C13)OC...,CN(C1=CC=CC=C1CNC2=NC(=NC=C2C(F)(F)F)NC3=CC4=C...,0
...,...,...,...,...,...,...,...
125543,957054-30-7,Selumetinib,VM-CUB-1,909780,CS(=O)(=O)N1CCN(CC1)CC2=CC3=C(S2)C(=NC(=N3)C4=...,CN1C=NC2=C1C=C(C(=C2F)NC3=C(C=C(C=C3)Br)Cl)C(=...,1
125544,AZD1208,Vorinostat,VM-CUB-1,909780,C1CC(CN(C1)C2=C(C=CC=C2C=C3C(=O)NC(=O)S3)C4=CC...,C1=CC=C(C=C1)NC(=O)CCCCCCC(=O)NO,0
125545,Saracatinib,AZD4320,VM-CUB-1,909780,CN1CCN(CC1)CCOC2=CC(=C3C(=C2)N=CN=C3NC4=C(C=CC...,CN(CCC(CSC1=CC=CC=C1)NC2=C(C=C(C=C2)S(=O)(=O)N...,0
125546,Saracatinib,AZD5582,VM-CUB-1,909780,CN1CCN(CC1)CCOC2=CC(=C3C(=C2)N=CN=C3NC4=C(C=CC...,CC(C(=O)NC(C1CCCCC1)C(=O)N2CCCC2C(=O)NC3C(CC4=...,1


In [19]:
## This file is to leav a cell out partition
def data_partition_leave_cells_out(num_folds,random_state,valid_set_portion,path_to_save, feature ):
    import pickle
    with open(targetdata_dir_raw +'/data_pairs.pkl', 'rb') as file:
        data_pairs = pickle.load(file )
        
    #grouped = data_pairs.groupby('Cosmic_ID')['Y'].value_counts(normalize=True).unstack(fill_value=0)
    grouped = data_pairs.groupby(feature)['Y'].value_counts(normalize=True).unstack(fill_value=0)
    ### we lable those have mostly 0 as class0, blanced 0 and 1 as class 1 and similar distribution to 
    ## the original class distribution, around 0.81 as 2 and more balanced ones are 3
    label = []
    for i in range(len(grouped)):
        if grouped[0].iloc[i]>=0.90:
            label.append(0)
        elif grouped[0].iloc[i]<0.2:
            label.append(1)
        elif grouped[0].iloc[i]<0.90 and grouped[0].iloc[i]>0.75:
            label.append(2)
        else:
            label.append(3)   
            
    grouped['label'] = label
    grouped = grouped.reset_index()
    
    X = grouped[feature]
    Y = grouped['label']
    
    skf_trte = StratifiedKFold(n_splits=num_folds, random_state=random_state, shuffle=True)  # split train and test
    skf_trv = StratifiedShuffleSplit(n_splits=2, 
                                     test_size=valid_set_portion, 
                                     random_state=random_state)  # split train and validation

    data_partition = {}
    fold_num = 0
    
    for train_index, test_index in skf_trte.split(X,Y):
        x_tr = np.zeros(len(train_index))
        y_tr = Y[train_index] 
        #print('train cell line number', len(train_index))
        print('test', feature,  'number', len(test_index))

        for tr_index, val_index in skf_trv.split(x_tr, y_tr):
            tr_ids = train_index[tr_index]
            val_ids = train_index[val_index]
            print('train', feature ,' number', len(tr_ids))
            print('validation', feature,  'number' , len(val_ids))
            
            data_partition[fold_num] = {'train': tr_ids,
                                             'validation': val_ids,
                                             'test': test_index}
            
            
            
        fold_num = fold_num + 1
    
    
    data_partitions = {}
   
    YY = data_pairs.Y
    for i in range(num_folds):
        train_cell = X[data_partition[i]['train']]
        val_cell = X[data_partition[i]['validation']]
        test_cell = X[data_partition[i]['test']]
        train_indices = np.array(data_pairs.loc[data_pairs[feature].isin(train_cell)].index)
        val_indices = np.array(data_pairs.loc[data_pairs[feature].isin(val_cell)].index)
        test_indices = np.array(data_pairs.loc[data_pairs[feature].isin(test_cell)].index)
        data_partitions[i] = {'train': train_indices,
                                             'validation': val_indices,
                                             'test': test_indices}
        
    
    
        print("fold_num:", i)
        print('train data')
        report_label_distrib(YY[data_partitions[i]['train']])
        print('validation data')
        report_label_distrib(YY[data_partitions[i]['validation']])
        print('test data')
        report_label_distrib(YY[data_partitions[i]['test']])
        print()
        fold_num += 1
        print("-"*25)
    
    file_name = path_to_save + f'/partions_{feature}_out.pkl'
    with open(file_name, "wb") as pickle_file:
        pickle.dump(data_partitions, pickle_file)
    return data_partitions


def report_label_distrib(labels):
    classes, counts = np.unique(labels, return_counts=True)
    norm_counts = counts/counts.sum()
    for i, label in enumerate(classes):
        print("class:", label, "norm count:", norm_counts[i])


def validate_partitioan(train_indices, val_indices, test_indices):
    # Find the intersection between test_indices and val_indices
    test_val_intersection = set(test_indices).intersection(val_indices)

    # Find the intersection between test_indices and train_indices
    test_train_intersection = set(test_indices).intersection(train_indices)

    # Find the intersection between val_indices and train_indices
    val_train_intersection = set(val_indices).intersection(train_indices)

    # Check if any of the intersections have common elements
    if test_val_intersection or test_train_intersection or val_train_intersection:
        print("The sets of indices intersect.")
    else:
        print("The sets of indices do not intersect.")

    # Calculate the total count of indices
    total_indices_count = len(test_indices) + len(val_indices) + len(train_indices)

    # Print the total count of indices
    print("Total count of indices:", total_indices_count)
    

In [20]:
#data_partitions =data_partition_leave_cells_out(5, 42, 0.1, targetdata_dir_processed,'Drug1_ID' )
partition_style = 'Drug1_ID'  #'Drug1_ID'  #'Cell_Line_ID'
data_partitions =data_partition_leave_cells_out(5, 42, 0.1, targetdata_dir_processed,partition_style )

test Drug1_ID number 368
train Drug1_ID  number 1322
validation Drug1_ID number 147
train Drug1_ID  number 1322
validation Drug1_ID number 147
test Drug1_ID number 368
train Drug1_ID  number 1322
validation Drug1_ID number 147
train Drug1_ID  number 1322
validation Drug1_ID number 147
test Drug1_ID number 367
train Drug1_ID  number 1323
validation Drug1_ID number 147
train Drug1_ID  number 1323
validation Drug1_ID number 147
test Drug1_ID number 367
train Drug1_ID  number 1323
validation Drug1_ID number 147
train Drug1_ID  number 1323
validation Drug1_ID number 147
test Drug1_ID number 367
train Drug1_ID  number 1323
validation Drug1_ID number 147
train Drug1_ID  number 1323
validation Drug1_ID number 147
fold_num: 0
train data
class: 0 norm count: 0.5003300931090218
class: 1 norm count: 0.49966990689097823
validation data
class: 0 norm count: 0.5151309335144517
class: 1 norm count: 0.4848690664855483
test data
class: 0 norm count: 0.518389723029088
class: 1 norm count: 0.4816102769709

In [46]:
#used_dataset = dataset
# If you want to use a smaller subset of the dataset for testing
#smaller_dataset_len = int(len(dataset)/100)
#used_dataset = dataset[:smaller_dataset_len]

#dataset = used_dataset

In [47]:
'''
fold_partitions = get_stratified_partitions(dataset.data.y,
                                            num_folds=5,
                                            valid_set_portion=0.1,
                                            random_state=42)
                                            
'''

'\nfold_partitions = get_stratified_partitions(dataset.data.y,\n                                            num_folds=5,\n                                            valid_set_portion=0.1,\n                                            random_state=42)\n                                            \n'

In [48]:
fold_partitions = data_partitions

In [49]:
print("Number of training graphs: "+ str(len(fold_partitions[0]['train'])))
print("Number of validation graphs: "+ str(len(fold_partitions[0]['validation'])))
print("Number of testing graphs: "+ str(len(fold_partitions[0]['test'])))

Number of training graphs: 81057
Number of validation graphs: 2551
Number of testing graphs: 24951


In [50]:
# training parameters

tp = {
    "batch_size" : 300,
    "num_epochs" : 100,
    
    "emb_dim" : 100,
    "gnn_type" : "gatv2",
    "num_layer" : 5,
    "graph_pooling" : "mean", #attention
    
    "input_embed_dim" : None,
    "gene_embed_dim": 1,
    "num_attn_heads" : 2,
    "num_transformer_units" : 1,
    "p_dropout" : 0.3,
    "nonlin_func" : 'nn.ReLU()',
    "mlp_embed_factor" : 2,
    "pooling_mode" : 'attn',
    "dist_opt" : 'cosine',

    "base_lr" : 3e-4, #3e-4
    "max_lr_mul": 10,
    "l2_reg" : 1e-3,  #"l2_reg" : 1e-7,
    "loss_w" : 1.,
    "margin_v" : 1.,

    "expression_dim" : 64,
    "expression_input_size" : 908,
    "exp_H1" : 4096,
    "exp_H2" : 1024
}

In [51]:

#q_i = 0
#partition = fold_partitions[q_i]
partition = fold_partitions
gpu_num = 1
time_stamp = datetime.datetime.now().strftime('%Y-%m-%d')
import torch.multiprocessing as mp
mp.set_start_method("spawn", force=True)
queue = mp.Queue()

for i in range(len(fold_partitions)):
    partition = fold_partitions[i]
    exp_dir = create_directory(os.path.join(targetdata_dir_exp, str(partition_style)+"_fold_"+str(i)+"_"+time_stamp))
    create_directory(os.path.join(exp_dir, "predictions"))
    create_directory(os.path.join(exp_dir, "modelstates"))
    deepadr.train_functions.run_exp(queue, dataset, 0, tp, exp_dir, partition)
print("End: " + datetime.datetime.now().strftime('%Y-%m-%d')) 


gpu: cuda:0
{'batch_size': 300, 'num_epochs': 100, 'emb_dim': 100, 'gnn_type': 'gatv2', 'num_layer': 5, 'graph_pooling': 'mean', 'input_embed_dim': None, 'gene_embed_dim': 1, 'num_attn_heads': 2, 'num_transformer_units': 1, 'p_dropout': 0.3, 'nonlin_func': 'nn.ReLU()', 'mlp_embed_factor': 2, 'pooling_mode': 'attn', 'dist_opt': 'cosine', 'base_lr': 0.0003, 'max_lr_mul': 10, 'l2_reg': 0.001, 'loss_w': 1.0, 'margin_v': 1.0, 'expression_dim': 64, 'expression_input_size': 908, 'exp_H1': 4096, 'exp_H2': 1024}


/home/ubuntu/anaconda3/envs/bedict_crispr/lib/python3.10/site-packages/torch_geometric/data/in_memory_dataset.py:157: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


Dropout(p=0.3, inplace=False) Dropout(p=0.3, inplace=False)
=====Epoch 0
Training...


Iteration:   3%|█                               | 9/271 [00:02<01:09,  3.79it/s]


KeyboardInterrupt: 

In [ ]:
#deepadr.train_functions.run_exp(queue, used_dataset, gpu_num, tp, exp_dir, partition)

In [ ]:
### muti-processing
time_stamp = datetime.datetime.now().strftime('%Y-%m-%d')
import torch.multiprocessing as mp
mp.set_start_method("spawn", force=True)
queue = mp.Queue()
q_processes = []
def spawn_q_process(q_process):
    print(">>> spawning hyperparam search process")
    q_process.start()
    
def join_q_process(q_process):
    q_process.join()
    print("<<< joined hyperparam search process")
    
def create_q_process(queue, dataset, gpu_num, tphp, exp_dir, partition):
#     fold_gpu_map = {0:gpu_num}
    return mp.Process(target=deepadr.train_functions.run_exp, args=(queue, dataset, gpu_num, tphp, exp_dir, partition))

print("Start: " + time_stamp)
for q_i in range(min(n_gpu, len(fold_partitions))):
    partition = fold_partitions[q_i]
    print(q_i)
    exp_dir = create_directory(os.path.join(targetdata_dir_exp, "fold_"+str(q_i)+"_"+time_stamp))
    create_directory(os.path.join(exp_dir, "predictions"))
    create_directory(os.path.join(exp_dir, "modelstates"))
    q_process = create_q_process(queue, dataset, q_i, tp, exp_dir, partition)
    q_processes.append(q_process)
    spawn_q_process(q_process)

spawned_processes = n_gpu
    
for q_i in range(min(n_gpu, len(fold_partitions))):
    join_q_process(q_processes[q_i])
    released_gpu_num = queue.get()
    print("released_gpu_num:", released_gpu_num)
    
print("End: " + datetime.datetime.now().strftime('%Y-%m-%d'))